In [1]:
import numpy as np
import pandas as pd
#import geopandas as gpd
#from shapely.geometry import Point
import os
import tensorflow as tf
from tqdm import tqdm
#from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta

from tensorflow.keras import layers
from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

for dirname, _, filenames in os.walk('./'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./torch_regression.py
./covid-19-rnn.ipynb
./seir-hcd-model.ipynb
./covid-19-forecasting-with-an-rnn-2.ipynb
./best_model.h5
./covid_rf_model.ipynb
./RNN.ipynb
./submission/sub.csv
./proc_data/train.csv
./proc_data/test.csv
./proc_data/SampleSubmission.csv
./raw_data/Population density by countries.csv
./raw_data/train.csv
./raw_data/SampleSubmission.csv
./raw_data/population_by_country_2020.csv
./.ipynb_checkpoints/RNN-checkpoint.ipynb
./.ipynb_checkpoints/covid-19-rnn-checkpoint.ipynb
./.ipynb_checkpoints/covid_rf_model-checkpoint.ipynb
./.ipynb_checkpoints/covid-19-forecasting-with-an-rnn-2-checkpoint.ipynb


The Training data (in train_df) is from 22 January to 5 March 2020 while the test data ranges from 6 March to 7 June 2020

In [22]:
train_df = pd.read_csv("./raw_data/train.csv")
test_df = pd.read_csv("./proc_data/test.csv")
train_df["Territory"] = [ter.replace("'","") for
                         ter in train_df["Territory"]]

# Cast Date to datetime
train_df['Date'] = pd.to_datetime(train_df['Date'])

train_df.head()

,Territory X Date,target,cases,Territory,Date
0,Afghanistan X 1/22/20,0,0,Afghanistan,2020-01-22
1,Afghanistan X 1/23/20,0,0,Afghanistan,2020-01-23
2,Afghanistan X 1/24/20,0,0,Afghanistan,2020-01-24
3,Afghanistan X 1/25/20,0,0,Afghanistan,2020-01-25
4,Afghanistan X 1/26/20,0,0,Afghanistan,2020-01-26


In [23]:
test_df.head(3)

,Territory X Date,target,Territory,Date
0,Afghanistan X 3/6/20,0,Afghanistan,2020-03-06
1,Afghanistan X 3/7/20,0,Afghanistan,2020-03-07
2,Afghanistan X 3/8/20,0,Afghanistan,2020-03-08


In [24]:
group_data = train_df.groupby('Territory')

In [35]:
for i in group_data.groups:
    print(group_data.get_group(i))

         Territory X Date  target  cases    Territory       Date
0   Afghanistan X 1/22/20       0      0  Afghanistan 2020-01-22
1   Afghanistan X 1/23/20       0      0  Afghanistan 2020-01-23
2   Afghanistan X 1/24/20       0      0  Afghanistan 2020-01-24
3   Afghanistan X 1/25/20       0      0  Afghanistan 2020-01-25
4   Afghanistan X 1/26/20       0      0  Afghanistan 2020-01-26
5   Afghanistan X 1/27/20       0      0  Afghanistan 2020-01-27
6   Afghanistan X 1/28/20       0      0  Afghanistan 2020-01-28
7   Afghanistan X 1/29/20       0      0  Afghanistan 2020-01-29
8   Afghanistan X 1/30/20       0      0  Afghanistan 2020-01-30
9   Afghanistan X 1/31/20       0      0  Afghanistan 2020-01-31
10   Afghanistan X 2/1/20       0      0  Afghanistan 2020-02-01
11   Afghanistan X 2/2/20       0      0  Afghanistan 2020-02-02
12   Afghanistan X 2/3/20       0      0  Afghanistan 2020-02-03
13   Afghanistan X 2/4/20       0      0  Afghanistan 2020-02-04
14   Afghanistan X 2/5/20

[71 rows x 5 columns]
                      Territory X Date  target  cases               Territory  \
1633  Bosnia and Herzegovina X 1/22/20       0      0  Bosnia and Herzegovina   
1634  Bosnia and Herzegovina X 1/23/20       0      0  Bosnia and Herzegovina   
1635  Bosnia and Herzegovina X 1/24/20       0      0  Bosnia and Herzegovina   
1636  Bosnia and Herzegovina X 1/25/20       0      0  Bosnia and Herzegovina   
1637  Bosnia and Herzegovina X 1/26/20       0      0  Bosnia and Herzegovina   
1638  Bosnia and Herzegovina X 1/27/20       0      0  Bosnia and Herzegovina   
1639  Bosnia and Herzegovina X 1/28/20       0      0  Bosnia and Herzegovina   
1640  Bosnia and Herzegovina X 1/29/20       0      0  Bosnia and Herzegovina   
1641  Bosnia and Herzegovina X 1/30/20       0      0  Bosnia and Herzegovina   
1642  Bosnia and Herzegovina X 1/31/20       0      0  Bosnia and Herzegovina   
1643   Bosnia and Herzegovina X 2/1/20       0      0  Bosnia and Herzegovina   
1644  

           Territory X Date  target  cases    Territory       Date
2911  Congo (the) X 1/22/20       0      0  Congo (the) 2020-01-22
2912  Congo (the) X 1/23/20       0      0  Congo (the) 2020-01-23
2913  Congo (the) X 1/24/20       0      0  Congo (the) 2020-01-24
2914  Congo (the) X 1/25/20       0      0  Congo (the) 2020-01-25
2915  Congo (the) X 1/26/20       0      0  Congo (the) 2020-01-26
2916  Congo (the) X 1/27/20       0      0  Congo (the) 2020-01-27
2917  Congo (the) X 1/28/20       0      0  Congo (the) 2020-01-28
2918  Congo (the) X 1/29/20       0      0  Congo (the) 2020-01-29
2919  Congo (the) X 1/30/20       0      0  Congo (the) 2020-01-30
2920  Congo (the) X 1/31/20       0      0  Congo (the) 2020-01-31
2921   Congo (the) X 2/1/20       0      0  Congo (the) 2020-02-01
2922   Congo (the) X 2/2/20       0      0  Congo (the) 2020-02-02
2923   Congo (the) X 2/3/20       0      0  Congo (the) 2020-02-03
2924   Congo (the) X 2/4/20       0      0  Congo (the) 2020-0

[71 rows x 5 columns]
                 Territory X Date  target  cases          Territory       Date
4118  Equatorial Guinea X 1/22/20       0      0  Equatorial Guinea 2020-01-22
4119  Equatorial Guinea X 1/23/20       0      0  Equatorial Guinea 2020-01-23
4120  Equatorial Guinea X 1/24/20       0      0  Equatorial Guinea 2020-01-24
4121  Equatorial Guinea X 1/25/20       0      0  Equatorial Guinea 2020-01-25
4122  Equatorial Guinea X 1/26/20       0      0  Equatorial Guinea 2020-01-26
4123  Equatorial Guinea X 1/27/20       0      0  Equatorial Guinea 2020-01-27
4124  Equatorial Guinea X 1/28/20       0      0  Equatorial Guinea 2020-01-28
4125  Equatorial Guinea X 1/29/20       0      0  Equatorial Guinea 2020-01-29
4126  Equatorial Guinea X 1/30/20       0      0  Equatorial Guinea 2020-01-30
4127  Equatorial Guinea X 1/31/20       0      0  Equatorial Guinea 2020-01-31
4128   Equatorial Guinea X 2/1/20       0      0  Equatorial Guinea 2020-02-01
4129   Equatorial Guinea X 2/2

[71 rows x 5 columns]
         Territory X Date  target  cases  Territory       Date
5538  Guatemala X 1/22/20       0      0  Guatemala 2020-01-22
5539  Guatemala X 1/23/20       0      0  Guatemala 2020-01-23
5540  Guatemala X 1/24/20       0      0  Guatemala 2020-01-24
5541  Guatemala X 1/25/20       0      0  Guatemala 2020-01-25
5542  Guatemala X 1/26/20       0      0  Guatemala 2020-01-26
5543  Guatemala X 1/27/20       0      0  Guatemala 2020-01-27
5544  Guatemala X 1/28/20       0      0  Guatemala 2020-01-28
5545  Guatemala X 1/29/20       0      0  Guatemala 2020-01-29
5546  Guatemala X 1/30/20       0      0  Guatemala 2020-01-30
5547  Guatemala X 1/31/20       0      0  Guatemala 2020-01-31
5548   Guatemala X 2/1/20       0      0  Guatemala 2020-02-01
5549   Guatemala X 2/2/20       0      0  Guatemala 2020-02-02
5550   Guatemala X 2/3/20       0      0  Guatemala 2020-02-03
5551   Guatemala X 2/4/20       0      0  Guatemala 2020-02-04
5552   Guatemala X 2/5/20       0

...

[71 rows x 5 columns]
                      Territory X Date  target  cases               Territory  \
8236  Marshall Islands (the) X 1/22/20       0      0  Marshall Islands (the)   
8237  Marshall Islands (the) X 1/23/20       0      0  Marshall Islands (the)   
8238  Marshall Islands (the) X 1/24/20       0      0  Marshall Islands (the)   
8239  Marshall Islands (the) X 1/25/20       0      0  Marshall Islands (the)   
8240  Marshall Islands (the) X 1/26/20       0      0  Marshall Islands (the)   
8241  Marshall Islands (the) X 1/27/20       0      0  Marshall Islands (the)   
8242  Marshall Islands (the) X 1/28/20       0      0  Marshall Islands (the)   
8243  Marshall Islands (the) X 1/29/20       0      0  Marshall Islands (the)   
8244  Marshall Islands (the) X 1/30/20       0      0  Marshall Islands (the)   
8245  Marshall Islands (the) X 1/31/20       0      0  Marshall Islands (the)   
8246   Marshall Islands (the) X 2/1/20       0      0  Marshall Islands (the)   
8247  

[71 rows x 5 columns]
               Territory X Date  target  cases        Territory       Date
9798  North Macedonia X 1/22/20       0      0  North Macedonia 2020-01-22
9799  North Macedonia X 1/23/20       0      0  North Macedonia 2020-01-23
9800  North Macedonia X 1/24/20       0      0  North Macedonia 2020-01-24
9801  North Macedonia X 1/25/20       0      0  North Macedonia 2020-01-25
9802  North Macedonia X 1/26/20       0      0  North Macedonia 2020-01-26
9803  North Macedonia X 1/27/20       0      0  North Macedonia 2020-01-27
9804  North Macedonia X 1/28/20       0      0  North Macedonia 2020-01-28
9805  North Macedonia X 1/29/20       0      0  North Macedonia 2020-01-29
9806  North Macedonia X 1/30/20       0      0  North Macedonia 2020-01-30
9807  North Macedonia X 1/31/20       0      0  North Macedonia 2020-01-31
9808   North Macedonia X 2/1/20       0      0  North Macedonia 2020-02-01
9809   North Macedonia X 2/2/20       0      0  North Macedonia 2020-02-02
981

                          Territory X Date  target  cases  \
10792  Republic of Moldova (the) X 1/22/20       0      0   
10793  Republic of Moldova (the) X 1/23/20       0      0   
10794  Republic of Moldova (the) X 1/24/20       0      0   
10795  Republic of Moldova (the) X 1/25/20       0      0   
10796  Republic of Moldova (the) X 1/26/20       0      0   
10797  Republic of Moldova (the) X 1/27/20       0      0   
10798  Republic of Moldova (the) X 1/28/20       0      0   
10799  Republic of Moldova (the) X 1/29/20       0      0   
10800  Republic of Moldova (the) X 1/30/20       0      0   
10801  Republic of Moldova (the) X 1/31/20       0      0   
10802   Republic of Moldova (the) X 2/1/20       0      0   
10803   Republic of Moldova (the) X 2/2/20       0      0   
10804   Republic of Moldova (the) X 2/3/20       0      0   
10805   Republic of Moldova (the) X 2/4/20       0      0   
10806   Republic of Moldova (the) X 2/5/20       0      0   
10807   Republic of Mold

          Territory X Date  target  cases  Territory       Date
11999  Singapore X 1/22/20       0      0  Singapore 2020-01-22
12000  Singapore X 1/23/20       0      1  Singapore 2020-01-23
12001  Singapore X 1/24/20       0      3  Singapore 2020-01-24
12002  Singapore X 1/25/20       0      3  Singapore 2020-01-25
12003  Singapore X 1/26/20       0      4  Singapore 2020-01-26
12004  Singapore X 1/27/20       0      5  Singapore 2020-01-27
12005  Singapore X 1/28/20       0      7  Singapore 2020-01-28
12006  Singapore X 1/29/20       0      7  Singapore 2020-01-29
12007  Singapore X 1/30/20       0     10  Singapore 2020-01-30
12008  Singapore X 1/31/20       0     13  Singapore 2020-01-31
12009   Singapore X 2/1/20       0     16  Singapore 2020-02-01
12010   Singapore X 2/2/20       0     18  Singapore 2020-02-02
12011   Singapore X 2/3/20       0     18  Singapore 2020-02-03
12012   Singapore X 2/4/20       0     24  Singapore 2020-02-04
12013   Singapore X 2/5/20       0     2

[71 rows x 5 columns]
       Territory X Date  target  cases Territory       Date
13845  Uganda X 1/22/20       0      0    Uganda 2020-01-22
13846  Uganda X 1/23/20       0      0    Uganda 2020-01-23
13847  Uganda X 1/24/20       0      0    Uganda 2020-01-24
13848  Uganda X 1/25/20       0      0    Uganda 2020-01-25
13849  Uganda X 1/26/20       0      0    Uganda 2020-01-26
13850  Uganda X 1/27/20       0      0    Uganda 2020-01-27
13851  Uganda X 1/28/20       0      0    Uganda 2020-01-28
13852  Uganda X 1/29/20       0      0    Uganda 2020-01-29
13853  Uganda X 1/30/20       0      0    Uganda 2020-01-30
13854  Uganda X 1/31/20       0      0    Uganda 2020-01-31
13855   Uganda X 2/1/20       0      0    Uganda 2020-02-01
13856   Uganda X 2/2/20       0      0    Uganda 2020-02-02
13857   Uganda X 2/3/20       0      0    Uganda 2020-02-03
13858   Uganda X 2/4/20       0      0    Uganda 2020-02-04
13859   Uganda X 2/5/20       0      0    Uganda 2020-02-05
13860   Uganda X 2

In [39]:
dataset = np.asarray(train_df.target, dtype='int32')
dataset = tf.constant(dataset)

In [42]:
# Creating a tensor from the numpy array
death_toll = tf.data.Dataset.from_tensor_slices(dataset)

In [49]:
# Windowing the death toll values
death_toll = death_toll.window(14, shift=1, drop_remainder=True)

# Creating a single dataset for of individual datasets in each window
death_toll = death_toll.flat_map(lambda window: window.batch(14))

# Batch creation
death_toll = death_toll.batch(32).prefetch(1)



AttributeError: in converted code:

    <ipython-input-45-e4254de1b782>:2 None  *
        death_toll = death_toll.flat_map(lambda window: window.batch(14))

    AttributeError: '_NestedVariant' object has no attribute 'batch'
